# Data Engineering Overview

The data used in this project flows through three SQLite databases:

![Data Flow Diagram](resources/BMBDataFlow.png)

* **<font color='red'>Web Data</font>** is collected in the **<font color='red'>Ingestion Database</font>** before being moved to more structured databases downstream.  The exception to this rule is basic **<font color='#008800'>Film Information</font>**, such as genres and descriptions, which are provided by the  TMDb API and loaded directly into the **<font color='#008800'>Processing Database</font>** as needed.

* The **<font color='blue'>Personal Movie Database</font>** contains any information unique to me, like whether I've seen a film, my opinion of it, and any tags it has been given.

* Any algorithmically-suggested labels (such as $Crime + Mystery + 1940's = Film\,Noir$) are loaded as **<font color='#008800'>Suggested Tags</font>** into the **<font color='#008800'>Processing Database</font>**.  After being verified or rejected, the are considered **<font color='blue'>Confirmed Tags</font>** and moved to the **<font color='blue'>Personal Movie Database</font>**, where they can be used to refine the labelling algorithms.

Each of the databases can be accessed through its own Python class.  These classes all inherit from the same `SQLiteDB` base class.

In [2]:
from bmb import IngestionDB, ProcessingDB, WarehouseDB

ingestion  = IngestionDB()
processing = ProcessingDB()
warehouse  = WarehouseDB()

## Database Tables

In [19]:
def database_report( db):
    # Define SQL Queries for later use.
    ALL_TABLES  = "SELECT DISTINCT tbl_name FROM sqlite_schema"
    GET_COLUMNS = lambda table : f"PRAGMA table_info({table})"
    ROW_COUNT   = lambda tabld: f"SELECT COUNT(*) FROM {table}"
    
    lines = [ db.__class__.__name__]
    lines.append( "="*len( lines[0]) )
    
    for table in db.get( ALL_TABLES):
        if table != "sqlite_sequence":
            num_cols = len( db.get( GET_COLUMNS( table )))
            num_rows = db.get( ROW_COUNT( table), single=True)
            lines.append( f"     {table} ({num_rows} rows x {num_cols} columns)")
    
    print( "\n".join( lines))
    
for db in (ingestion, processing, warehouse):
    database_report( db)

IngestionDB
     Streamer (3 rows x 3 columns)
     JustwatchFilm (1897 rows x 5 columns)
     JustwatchState (1929 rows x 5 columns)
     TMDbFilm (4488 rows x 4 columns)
     TMDbRecommendation (18229 rows x 2 columns)
ProcessingDB
     FilmGenre (22639 rows x 2 columns)
     FilmDescription (9753 rows x 2 columns)
     FilmCredit (0 rows x 2 columns)
     OldRating (9024 rows x 3 columns)
     Film (11427 rows x 5 columns)
     Genre (19 rows x 2 columns)
     Credit (0 rows x 2 columns)
     Status (5 rows x 3 columns)
     Tag (0 rows x 3 columns)
     FilmTag (0 rows x 3 columns)
     Metatag (0 rows x 3 columns)
     GenreOrdering (19 rows x 3 columns)
WarehouseDB
     FilmGenre (1237 rows x 2 columns)
     FilmDescription (462 rows x 2 columns)
     FilmCredit (0 rows x 2 columns)
     Genre (19 rows x 2 columns)
     Credit (0 rows x 2 columns)
     Tag (0 rows x 3 columns)
     FilmTag (0 rows x 3 columns)
     Metatag (0 rows x 3 columns)
     Film (462 rows x 4 columns)
